# DL LAB ASSIGNMENT - 4 
## CS22B1096 PRATYEK THUMULA
- Topic: Binary Classification of Skin Lesions
- Date: 12/02/2025

### IMPORTING MODULES

In [2]:
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from sklearn.metrics import classification_report

### UPLOADING DATASET AND BUILDING ARCHITECTURE

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

class SkinLesionDataset(Dataset):
    def __init__(self, csv_file, root, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root = root
        self.transform = transform
        self.final_data = self.get_data()

    def get_data(self):
        data = []
        for i, row in self.data.iterrows():
            img_name = row['image_name'] + ".jpg"
            label = row['target']

            image_path = os.path.join(self.root, img_name)
            if os.path.exists(image_path):
                data.append((image_path, label))

        return data

    def __len__(self):
        return len(self.final_data)

    def __getitem__(self, index):
        image_path, label = self.final_data[index]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, label

root = "/kaggle/input/lab4-dl/isic2020/"
train_dataset = SkinLesionDataset(csv_file = os.path.join(root, "train.csv"), root=os.path.join(root, "train/"), transform=transform)
test_dataset = SkinLesionDataset(csv_file = os.path.join(root, "test.csv"), root=os.path.join(root, "test/"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class block(nn.Module):
    def __init__(self, in_channels, intermediate_channels, identity_downsample=None, stride=1):
        super(block, self).__init__()
        self.expansion = 4
        
        self.conv1 = nn.Conv2d(
            in_channels, 
            intermediate_channels, 
            kernel_size=1, 
            stride=1, 
            padding=0, 
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

class ResNet152(nn.Module):
    def __init__(self, block, num_classes):
        super(ResNet152, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(block, 3, intermediate_channels=64, stride=1)
        self.layer2 = self._make_layer(block, 8, intermediate_channels=128, stride=2)
        self.layer3 = self._make_layer(block, 36, intermediate_channels=256, stride=2)
        self.layer4 = self._make_layer(block, 3, intermediate_channels=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
    
    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []
        
        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        
        
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )
        
        layers.append(
        block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )
        
        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4
        
        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




In [9]:
model = ResNet152(block, num_classes=2)
model = nn.DataParallel(model) 
model.to(device)
print(model)

DataParallel(
  (module): ResNet152(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): block(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (identity_downsample): Sequential(
          (0): Conv2d(6

### TRAINING WITH ADAM OPTIMIZER AND 0.001 L.R

In [4]:
def train_model(model, train_loader, criterion, optimizer, epochs=2):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

        # Classification report after each epoch
        print("\nClassification Report:")
        print(classification_report(all_labels, all_preds, zero_division=0))

def test_model(model, test_loader):
    model.eval()  
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad(): 
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)  
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(test_loader)
    accuracy = 100 * correct / total
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")


criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, criterion, optimizer, epochs=2)
test_model(model, test_loader)


Epoch [1/2], Loss: 0.1005, Accuracy: 98.19%

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     29978
           1       0.00      0.00      0.00       541

    accuracy                           0.98     30519
   macro avg       0.49      0.50      0.50     30519
weighted avg       0.96      0.98      0.97     30519

Epoch [2/2], Loss: 0.0893, Accuracy: 98.21%

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     29978
           1       0.00      0.00      0.00       541

    accuracy                           0.98     30519
   macro avg       0.49      0.50      0.50     30519
weighted avg       0.96      0.98      0.97     30519

Test Loss: 0.1181, Test Accuracy: 98.17%


### TRAINING WITH ADAM AND 0.01 L.R


In [5]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

train_model(model, train_loader, criterion, optimizer, epochs=2)
test_model(model, test_loader)

Epoch [1/2], Loss: 0.1422, Accuracy: 98.14%

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     29978
           1       0.00      0.00      0.00       541

    accuracy                           0.98     30519
   macro avg       0.49      0.50      0.50     30519
weighted avg       0.96      0.98      0.97     30519

Epoch [2/2], Loss: 0.0883, Accuracy: 98.22%

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     29978
           1       0.00      0.00      0.00       541

    accuracy                           0.98     30519
   macro avg       0.49      0.50      0.50     30519
weighted avg       0.96      0.98      0.97     30519

Test Loss: 0.0930, Test Accuracy: 98.17%


### TRAINING WITH SGD AND 0.001 L.R

In [5]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

train_model(model, train_loader, criterion, optimizer, epochs=2)
test_model(model, test_loader)

Epoch [1/2], Loss: 0.0904, Accuracy: 98.23%

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     29978
           1       0.00      0.00      0.00       541

    accuracy                           0.98     30519
   macro avg       0.49      0.50      0.50     30519
weighted avg       0.96      0.98      0.97     30519

Epoch [2/2], Loss: 0.0873, Accuracy: 98.23%

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     29978
           1       0.00      0.00      0.00       541

    accuracy                           0.98     30519
   macro avg       0.49      0.50      0.50     30519
weighted avg       0.96      0.98      0.97     30519

Test Loss: 0.0956, Test Accuracy: 98.17%


### TRAINING WITH SGD AND 0.01 L.R

In [6]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

train_model(model, train_loader, criterion, optimizer, epochs=2)
test_model(model, test_loader)

Epoch [1/2], Loss: 0.1124, Accuracy: 97.98%

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     29978
           1       0.00      0.00      0.00       541

    accuracy                           0.98     30519
   macro avg       0.49      0.50      0.49     30519
weighted avg       0.96      0.98      0.97     30519

Epoch [2/2], Loss: 0.0907, Accuracy: 98.15%

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     29978
           1       0.00      0.00      0.00       541

    accuracy                           0.98     30519
   macro avg       0.49      0.50      0.50     30519
weighted avg       0.96      0.98      0.97     30519

Test Loss: 0.1142, Test Accuracy: 98.17%
